### Import Stuff

In [6]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions._
import scala.math.BigDecimal.RoundingMode

### Create a SQL Context

In [4]:
val sqlContext = new SQLContext(sc)


### Now some UDFs, because Spark 1.4 does not have all of the functions we need
The round function needs to take the type java.math.BigDecimal because that's what the dataframe created from the cassandraTable contains.  It does return the scala type through the scala implicits

In [7]:
val concat = udf((s1:String, s2:String) => s1 + s2)
val round = udf((f1:java.math.BigDecimal, places:Int) => f1.setScale(places,RoundingMode.HALF_EVEN))

### Create dataframes on the stores and receipts_by_store_date table

In [20]:
val stores_df = sqlContext.read.format("org.apache.spark.sql.cassandra").
      options(Map("keyspace"-> "retail", "table" -> "stores")).
      load()
      
val receipts_by_store_date_df = sqlContext.read.format("org.apache.spark.sql.cassandra").
      options(Map("keyspace"-> "retail", "table" -> "receipts_by_store_date")).
      load()
      

### Compute the sales_by_state
1. join receipts_by_store_date to store
2. group by state
3. sum by receipt_total
4. do a select to add the dummy column, rename columns, compute the region and round the totals

In [21]:
    val sales_by_state_df = receipts_by_store_date_df.
      join(stores_df, stores_df("store_id") === receipts_by_store_date_df("store_id")).
      groupBy(stores_df("state")).
      sum("receipt_total").
      select(lit("dummy") alias "dummy", col("state"), concat( lit("US-"), col("state")) alias "region", round(col("SUM(receipt_total)"), lit(2)) alias ("receipts_total"))

In [23]:
sales_by_state_df show 10

+-----+-----+------+--------------+
|dummy|state|region|receipts_total|
+-----+-----+------+--------------+
|dummy|   MS| US-MS|    5716738.79|
|dummy|   MT| US-MT|    1821736.04|
|dummy|   TN| US-TN|   10310875.43|
|dummy|   NC| US-NC|   12353711.41|
|dummy|   ND| US-ND|    2756610.09|
|dummy|   NH| US-NH|    1865259.78|
|dummy|   AL| US-AL|    5518584.73|
|dummy|   NJ| US-NJ|   11041570.96|
|dummy|   TX| US-TX|   24554911.09|
|dummy|   NM| US-NM|    2849598.96|
+-----+-----+------+--------------+



### Save it to sales_by_state.  First truncate the table

In [24]:
%%cql truncate retail.sales_by_state

In [25]:
sales_by_state_df.write.
      format("org.apache.spark.sql.cassandra").
      options(Map("keyspace" -> "retail",
                  "table" -> "sales_by_state")).
      save()

In [28]:
%%cql select * from retail.sales_by_state limit 5

dummy,receipts_total,state,region
dummy,28487231.46,FL,US-FL
dummy,24554911.09,TX,US-TX
dummy,20040746.96,PA,US-PA
dummy,19165754.08,CA,US-CA
dummy,15976686.76,NY,US-NY
